In [1]:
import pandas as pd
import numpy as np

In [2]:
activity_file   = 'data/activity_tracker_dataset.csv'
leave_file      = 'data/leave_dataset.csv'
onboarding_file = 'data/onboarding_dataset.csv'
performance_file= 'data/performance_dataset.csv'
rewards_file    = 'data/rewards_dataset.csv'
vibemeter_file  = 'data/vibemeter_dataset.csv'

file_info = {
    'activity': activity_file,
    'leave': leave_file,
    'onboarding': onboarding_file,
    'performance': performance_file,
    'rewards': rewards_file,
    'vibemeter': vibemeter_file
}

# 1. Summary statistics

## 1.1 Activity Df

In [157]:
activity_df=pd.read_csv(activity_file, parse_dates=['Date'])
activity_df.columns = activity_df.columns.str.strip()

In [158]:
activity_df.head()

,Employee_ID,Date,Teams_Messages_Sent,Emails_Sent,Meetings_Attended,Work_Hours
0,EMP0048,2023-01-01,33,8,9,6.69
1,EMP0207,2023-01-02,21,29,2,5.42
2,EMP0112,2023-01-03,21,1,3,9.21
3,EMP0309,2023-01-04,14,13,6,7.78
4,EMP0142,2023-01-05,18,3,0,5.80


In [159]:
activity_df.Employee_ID.nunique(), activity_df.Employee_ID.shape

(312, (500,))

In [160]:
activity_df.Date.min(), activity_df.Date.max(), activity_df.Date.nunique()

(Timestamp('2023-01-01 00:00:00'), Timestamp('2024-05-14 00:00:00'), 500)

> We should generate some statistics per employee basis (dataframe should be updated with each new information), drop time-series information for statistical analysis. These statistics can be generated with each new entry in any of the dataset.

In [161]:
activity_summary = activity_df.groupby('Employee_ID').agg({
    'Teams_Messages_Sent': ['sum', 'mean', 'median', 'std'], #Median - robust to outliers, std - variability - inconsistent schedules
    'Emails_Sent': ['sum', 'mean', 'median', 'std'],
    'Meetings_Attended': ['sum', 'mean', 'median', 'std'],
    'Work_Hours': ['sum', 'mean', 'median', 'std'],
    'Date': ['max', 'count']
}).reset_index()

std_columns = [col for col in activity_summary.columns if col[1].endswith('std')]
activity_summary[std_columns] = activity_summary[std_columns].fillna(0)

In [162]:
activity_summary.head()

Employee_ID Teams_Messages_Sent                         Emails_Sent        \
                              sum  mean median        std         sum  mean   
0     EMP0002                  75  37.5   37.5   0.707107          15   7.5   
1     EMP0003                  16  16.0   16.0   0.000000          28  28.0   
2     EMP0005                  27  13.5   13.5   3.535534          32  16.0   
3     EMP0006                  95  47.5   47.5   2.121320          34  17.0   
4     EMP0009                  44  22.0   22.0  31.112698          50  25.0   

                    Meetings_Attended                       Work_Hours         \
  median        std               sum mean median       std        sum   mean   
0    7.5   0.707107                15  7.5    7.5  2.121320      12.50  6.250   
1   28.0   0.000000                 2  2.0    2.0  0.000000       4.09  4.090   
2   16.0  16.970563                12  6.0    6.0  1.414214      14.75  7.375   
3   17.0   0.000000                11  5.5    5.5  0.707107      12.39  6.195   
4   25.0   1.414214                 2  1.0    1.0  1.414214      16.28  8.140   

                         Date        
  median       std        max count  
0  6.250  1.131371 2024-05-07     2  
1  4.090  0.000000 2023-12-07     1  
2  7.375  3.641600 2023-08-30     2  
3  6.195  0.982878 2024-01-29     2  
4  8.140  1.301076 2024-02-25     2

In [163]:
def flatten_columns(df):
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(map(str, col)).strip('_') for col in df.columns.values]
    return df
activity_summary = flatten_columns(activity_summary)

In [165]:
activity_summary.rename(columns={'Date_max': 'Last_activity_entry', 'Date_count': 'Total_activity_entry'}, inplace=True)
activity_summary.sample(5)

,Employee_ID,Teams_Messages_Sent_sum,Teams_Messages_Sent_mean,Teams_Messages_Sent_median,Teams_Messages_Sent_std,Emails_Sent_sum,Emails_Sent_mean,Emails_Sent_median,Emails_Sent_std,Meetings_Attended_sum,Meetings_Attended_mean,Meetings_Attended_median,Meetings_Attended_std,Work_Hours_sum,Work_Hours_mean,Work_Hours_median,Work_Hours_std,Last_activity_entry,Total_activity_entry
51,EMP0077,41,20.5,20.5,13.435029,26,13.0,13.0,9.899495,11,5.5,5.5,2.121320,18.94,9.470,9.470,0.664680,2023-11-27,2
41,EMP0064,46,46.0,46.0,0.000000,9,9.0,9.0,0.000000,3,3.0,3.0,0.000000,4.40,4.400,4.400,0.000000,2024-04-04,1
31,EMP0048,63,31.5,31.5,2.121320,16,8.0,8.0,0.000000,9,4.5,4.5,6.363961,15.39,7.695,7.695,1.421285,2023-08-22,2
275,EMP0438,35,35.0,35.0,0.000000,17,17.0,17.0,0.000000,7,7.0,7.0,0.000000,8.45,8.450,8.450,0.000000,2023-08-04,1
310,EMP0499,35,35.0,35.0,0.000000,6,6.0,6.0,0.000000,5,5.0,5.0,0.000000,5.51,5.510,5.510,0.000000,2024-04-05,1


## 1.2 Leave Df

In [ ]:
leave_df = pd.read_csv(leave_file, parse_dates=['Leave_Start_Date', 'Leave_End_Date'])
leave_df.columns = leave_df.columns.str.strip()

In [ ]:
reference_date = leave_df.Leave_End_Date.max() #Should be current date

In [47]:
leave_df['Delta_Days'] = (reference_date - leave_df['Leave_End_Date']).dt.days

In [67]:
lambda_decay = 0.001 #Tune

> $Leave\_Factor = (Leave\_Days)e^{–λ \cdot Δ\_days}$

In [61]:
leave_df['Leave_Factor'] = leave_df['Leave_Days'] * np.exp(-lambda_decay * leave_df['Delta_Days'])

In [62]:
leave_factor = leave_df.groupby(['Employee_ID', 'Leave_Type'])['Leave_Factor'].sum().reset_index()

In [63]:
leave_factor_pivot = leave_factor.pivot(index='Employee_ID', columns='Leave_Type', values='Leave_Factor').reset_index()

In [64]:
leave_factor_pivot = leave_factor_pivot.fillna(0)

In [65]:
leave_summary = leave_factor_pivot.rename(columns=lambda x: x + '_Factor' if x != 'Employee_ID' else x)

In [149]:
leave_summary.sample(5)

Leave_Type,Employee_ID,Annual Leave_Factor,Casual Leave_Factor,Sick Leave_Factor,Unpaid Leave_Factor
201,EMP0335,0.000000,0.000000,0.0,1.23633
302,EMP0491,2.615079,0.000000,0.0,0.00000
54,EMP0088,0.000000,9.252514,0.0,0.00000
202,EMP0336,0.000000,7.060989,0.0,0.00000
187,EMP0311,3.388185,3.594619,0.0,0.00000


## 1.3 Onboarding Df

In [150]:
onboarding_df = pd.read_csv(onboarding_file, parse_dates=['Joining_Date'])
onboarding_df.columns = onboarding_df.columns.str.strip()
onboarding_df['Onboarding_Feedback'] = onboarding_df['Onboarding_Feedback'].astype(str).str.strip()
onboarding_df['Mentor_Assigned'] = onboarding_df['Mentor_Assigned'].astype(str).str.strip().map({'True': True, 'False': False})
onboarding_df['Initial_Training_Completed'] = onboarding_df['Initial_Training_Completed'].astype(str).str.strip().map({'True': True, 'False': False})

In [151]:
onboarding_summary = onboarding_df.groupby('Employee_ID', as_index=False).agg({
    'Joining_Date': 'first',
    'Onboarding_Feedback': 'last',
    'Mentor_Assigned': 'last',
    'Initial_Training_Completed': 'last'
})

In [152]:
onboarding_summary.Onboarding_Feedback.unique()

array(['Excellent', 'Poor', 'Average', 'Good'], dtype=object)

In [153]:
feedback_mapping = {
    'Excellent': 3,
    'Poor': 0,
    'Average': 1,
    'Good': 2
}

onboarding_summary['Feedback_Score'] = onboarding_summary['Onboarding_Feedback'].map(feedback_mapping)

reference_date = onboarding_summary.Joining_Date.max()

onboarding_summary['Days_Since_Joining'] = (reference_date - onboarding_summary['Joining_Date']).dt.days

lambda_decay = 0.01
onboarding_summary['Decay_Factor'] = np.exp(-lambda_decay * onboarding_summary['Days_Since_Joining'])
onboarding_summary['Onboarding_Factor'] = onboarding_summary['Feedback_Score'] * onboarding_summary['Decay_Factor']
onboarding_summary.drop(['Decay_Factor', 'Feedback_Score'], axis=1, inplace=True)

In [154]:
onboarding_summary.head()

,Employee_ID,Joining_Date,Onboarding_Feedback,Mentor_Assigned,Initial_Training_Completed,Days_Since_Joining,Onboarding_Factor
0,EMP0001,2024-03-26,Excellent,True,False,49,1.837879
1,EMP0003,2023-11-29,Poor,True,True,167,0.000000
2,EMP0004,2023-01-11,Average,True,True,489,0.007521
3,EMP0006,2023-12-05,Good,False,False,161,0.399775
4,EMP0009,2023-08-31,Average,True,False,257,0.076536


## 1.4 Performance Df

In [89]:
perf_df = pd.read_csv(performance_file)

perf_df.columns = perf_df.columns.str.strip()

perf_df['Review_Period'] = perf_df['Review_Period'].astype(str).str.strip()
perf_df['Manager_Feedback'] = perf_df['Manager_Feedback'].astype(str).str.strip()

perf_df['Performance_Rating'] = pd.to_numeric(perf_df['Performance_Rating'], errors='coerce')

perf_df['Promotion_Consideration'] = perf_df['Promotion_Consideration'].astype(str).str.strip().map({
    'TRUE': True, 'FALSE': False, 'True': True, 'False': False
})

In [91]:
perf_df.head()

,Employee_ID,Review_Period,Performance_Rating,Manager_Feedback,Promotion_Consideration
0,EMP0220,Annual 2023,4,Meets Expectations,False
1,EMP0145,H2 2023,3,Exceeds Expectations,True
2,EMP0321,H1 2023,4,Meets Expectations,True
3,EMP0143,H2 2023,4,Needs Improvement,False
4,EMP0467,H2 2023,3,Exceeds Expectations,True


> Idea is to select the latest review, it is what matters!

In [101]:
perf_df[['Period', 'Year']] = perf_df['Review_Period'].str.split(' ', expand=True)
perf_df['Year'] = pd.to_numeric(perf_df['Year'], errors='coerce')

period_mapping = {'H1': 1, 'H2': 2, 'Annual': 3}
perf_df['Period_Order'] = perf_df['Period'].map(period_mapping)

perf_df_sorted = perf_df.sort_values(['Employee_ID', 'Year', 'Period_Order'], ascending=[True, False, False])

perf_summary = perf_df_sorted.groupby('Employee_ID', as_index=False).first()

perf_summary = perf_summary.drop(columns=['Review_Period', 'Period_Order']) #Drop them, since the last review matters, not the date.
perf_summary = perf_summary.rename(columns={'Period': 'Last_Review_Period', 'Year': 'Last_Review_Year'})
perf_summary.head()

,Employee_ID,Performance_Rating,Manager_Feedback,Promotion_Consideration,Last_Review_Period,Last_Review_Year
0,EMP0001,4,Meets Expectations,False,H1,2023
1,EMP0002,3,Exceeds Expectations,True,Annual,2023
2,EMP0003,4,Meets Expectations,True,Annual,2023
3,EMP0004,3,Meets Expectations,False,H2,2023
4,EMP0006,1,Exceeds Expectations,False,H1,2023


## 1.5 Rewards Df

In [102]:
rewards_df = pd.read_csv(rewards_file, parse_dates=['Award_Date'])

In [103]:
rewards_df.columns = rewards_df.columns.str.strip()
rewards_df['Award_Type'] = rewards_df['Award_Type'].astype(str).str.strip()
rewards_df['Reward_Points'] = pd.to_numeric(rewards_df['Reward_Points'], errors='coerce')

In [104]:
reference_date = rewards_df.Award_Date.max()

In [105]:
rewards_df['Days_Since_Award'] = (reference_date - rewards_df['Award_Date']).dt.days

In [106]:
lambda_decay = 0.01

In [107]:
rewards_df['Decayed_Reward_Points'] = rewards_df['Reward_Points'] * np.exp(-lambda_decay * rewards_df['Days_Since_Award'])

In [112]:
rewards_df['Decayed_Reward_Points'] = rewards_df['Reward_Points'] * np.exp(-lambda_decay * rewards_df['Days_Since_Award'])

reward_counts = rewards_df.groupby(['Employee_ID', 'Award_Type']).size().reset_index(name='Reward_Count')

reward_counts_pivot = reward_counts.pivot(index='Employee_ID', columns='Award_Type', values='Reward_Count').reset_index()
reward_counts_pivot = reward_counts_pivot.fillna(0)

reward_counts_pivot.columns = ['Employee_ID'] + [f"{col}_Count" for col in reward_counts_pivot.columns if col != 'Employee_ID']

decayed_points_total = rewards_df.groupby('Employee_ID')['Decayed_Reward_Points'].sum().reset_index()
decayed_points_total = decayed_points_total.rename(columns={'Decayed_Reward_Points': 'Total_Decayed_Reward_Points'})

rewards_summary = pd.merge(reward_counts_pivot, decayed_points_total, on='Employee_ID', how='outer')

In [119]:
rewards_summary.sample(5)

,Employee_ID,Best Team Player_Count,Innovation Award_Count,Leadership Excellence_Count,Star Performer_Count,Total_Decayed_Reward_Points
86,EMP0136,0.0,0.0,1.0,0.0,87.647439
182,EMP0280,0.0,0.0,1.0,0.0,14.591564
222,EMP0346,0.0,0.0,1.0,0.0,46.573814
174,EMP0269,1.0,0.0,1.0,0.0,29.655502
73,EMP0119,0.0,0.0,1.0,0.0,7.483131


## 1.6 Vibemeter Df

In [123]:
vibe_df = pd.read_csv(vibemeter_file, parse_dates=['Response_Date'])
vibe_df.columns = vibe_df.columns.str.strip()
vibe_df['Emotion_Zone'] = vibe_df['Emotion_Zone'].astype(str).str.strip()

In [124]:
vibe_df.Emotion_Zone.unique()

array(['Sad Zone', 'Leaning to Happy Zone', 'Neutral Zone (OK)',
       'Excited Zone', 'Happy Zone', 'Frustrated Zone',
       'Leaning to Sad Zone'], dtype=object)

In [125]:
emotion_mapping = {
    'Frustrated Zone': -3,
    'Sad Zone': -2,
    'Leaning to Sad Zone': -1,
    'Neutral Zone (OK)': 0,
    'Leaning to Happy Zone': 1,
    'Happy Zone': 2,
    'Excited Zone': 3
}
vibe_df['Emotion_Value'] = vibe_df['Emotion_Zone'].map(emotion_mapping)

In [126]:
reference_date = vibe_df.Response_Date.max()

In [137]:
vibe_df['Days_Since_Response'] = (reference_date - vibe_df['Response_Date']).dt.days

lambda_decay1, lambda_decay2 = 0.005, 0.005

vibe_df['Decayed_Emotion_Zone'] = vibe_df['Emotion_Value'] * np.exp(-lambda_decay1 * vibe_df['Days_Since_Response'])
vibe_df['Decayed_Vibe'] = vibe_df['Vibe_Score'] * np.exp(-lambda_decay2 * vibe_df['Days_Since_Response'])

vibe_summary = vibe_df.groupby('Employee_ID').agg({
    'Decayed_Emotion_Zone': 'sum',
    'Decayed_Vibe': 'sum'
}).reset_index()

In [138]:
vibe_summary.sample(5)

,Employee_ID,Decayed_Emotion_Zone,Decayed_Vibe
140,EMP0226,-1.542821,2.057094
16,EMP0022,0.623845,1.559613
272,EMP0437,-0.190651,1.772278
273,EMP0438,-0.702566,6.532014
36,EMP0051,-1.654688,2.757813


In [166]:
master_df = activity_summary.merge(leave_summary, on='Employee_ID', how='outer') \
                       .merge(onboarding_summary, on='Employee_ID', how='outer') \
                       .merge(perf_summary, on='Employee_ID', how='outer') \
                       .merge(rewards_summary, on='Employee_ID', how='outer') \
                       .merge(vibe_summary, on='Employee_ID', how='outer')

In [171]:
master_df = master_df.fillna(0)
master_df.head()

,Employee_ID,Teams_Messages_Sent_sum,Teams_Messages_Sent_mean,Teams_Messages_Sent_median,Teams_Messages_Sent_std,Emails_Sent_sum,Emails_Sent_mean,Emails_Sent_median,Emails_Sent_std,Meetings_Attended_sum,...,Promotion_Consideration,Last_Review_Period,Last_Review_Year,Best Team Player_Count,Innovation Award_Count,Leadership Excellence_Count,Star Performer_Count,Total_Decayed_Reward_Points,Decayed_Emotion_Zone,Decayed_Vibe
0,EMP0001,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,False,H1,2023.0,0.0,1.0,0.0,0.0,5.491134,0.000000,0.000000
1,EMP0002,75.0,37.5,37.5,0.707107,15.0,7.5,7.5,0.707107,15.0,...,True,Annual,2023.0,1.0,0.0,0.0,0.0,60.564878,-0.284681,0.094894
2,EMP0003,16.0,16.0,16.0,0.000000,28.0,28.0,28.0,0.000000,2.0,...,True,Annual,2023.0,0.0,0.0,1.0,1.0,71.517269,-1.134752,2.391022
3,EMP0004,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,...,False,H2,2023.0,1.0,0.0,1.0,1.0,21.712992,0.000000,2.615079
4,EMP0005,27.0,13.5,13.5,3.535534,32.0,16.0,16.0,16.970563,12.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.000000,-1.671379,2.263700


In [172]:
master_df.to_csv('data/clean/master_df.csv')